# Imports
---

In [1]:
import random
import networkx as nx
import tsplib95
from mst import MST
import matplotlib.pyplot as plt
import benchmark

# Helper Functions
---

In [2]:
def plot_graph(graph : nx.Graph, name : str):
    pos = nx.shell_layout(graph)
    nx.draw(graph, pos, with_labels=False)
    weights = {edge : str(graph.get_edge_data(edge[0], edge[1])["weight"]) for edge in graph.edges}
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=weights)
    plt.savefig(name)
    plt.close()
    
def tour_cost(tsp_tour, graph: nx.Graph):
    total_cost = 0
    for i in range(len(tsp_tour) - 1):
        u, v = tsp_tour[i], tsp_tour[i+1]
        total_cost += graph[u][v]['weight']
    return total_cost

def approximation_ratio(ALG_COST, OPT_COST):
    return ALG_COST / OPT_COST # Minimization problem

# Christofides Algorithm
---
Steps:
1. Find a minimum spanning tree T of G.
2. Let O be the set of vertices with odd degree in T.
3. Find a minimum weight perfect matching M in the induced subgraph given by the vertices from O.
4. Combine the edges of M and T to form a connected multigraph H in which each vertex has even degree.
5. Form an Eulerian circuit in H.
6. Make the circuit found in step 5 into a Hamiltonian circuit by skipping repeated vertices (shortcutting).


In [3]:
def christofides(graph: nx.Graph):
    # Step 1: Compute the Minimum Spanning Tree (MST) using Kruskal's Algorithm
    mst = MST()
    # mst_graph = mst.get_mst_kruskal_k(graph, k=20)  # Assuming an MST implementation using Kruskal's algorithm
    # mst_graph = mst.get_mst_kruskal(graph)
    mst_graph = mst.get_mst_prims_k(graph=graph, k=10)
    plot_graph(mst_graph, 'mst.png')

    # Step 2: Identify vertices with odd degree and create a subgraph
    odd_nodes = MST.get_odd_degree_nodes(mst_graph)
    subgraph = nx.subgraph(graph, odd_nodes)
    plot_graph(subgraph, name='subgraph.png')

    # Step 3: Find the minimum cost perfect matching using Blossom Algorithm (Shrinks odd-length cycles aka BLOSSOMS)
    # Using the Blossom algorithm via nx.max_weight_matching (-weights)
    new_graph = nx.Graph()
    for e in subgraph.edges:
        new_graph.add_edge(e[0], e[1], weight=-subgraph.get_edge_data(e[0], e[1])['weight'])

    set_matching = nx.max_weight_matching(new_graph, maxcardinality=True)
    print("Minimum Cost Perfect Matching Pairs:", set_matching)

    # Add the matching edges to create a multigraph
    matching_graph = nx.Graph()
    for m in set_matching:
        matching_graph.add_edge(m[0], m[1], weight=graph.get_edge_data(m[0], m[1])['weight'])

    plot_graph(matching_graph, name='matching_graph.png')

    # Step 4: Combine the MST and matching graph to form an Eulerian graph
    eulerian_graph = nx.MultiGraph(mst_graph)
    eulerian_graph.add_edges_from(matching_graph.edges(data=True))
    # plot_graph(eulerian_graph, name='eulerian_graph.png')

    # Step 5: Compute the Eulerian circuit and shortcut to form the TSP tour
    eulerian_circuit = list(nx.eulerian_circuit(eulerian_graph))
    tsp_tour = []
    visited = set()

    for u, v in eulerian_circuit:
        if u not in visited:
            tsp_tour.append(u)
            visited.add(u)

    tsp_tour.append(tsp_tour[0])  # Complete the tour by returning to the start

    # Step 6: Removing shortcuts to form the TSP tour using traingle inequality (Hamiltionian cycle)
    for i in range(len(tsp_tour) - 1):
        a, b, c = tsp_tour[i - 1], tsp_tour[i], tsp_tour[(i + 1) % len(tsp_tour)]
        if graph.has_edge(a, c) and graph[a][c]['weight'] <= graph[a][b]['weight'] + graph[b][c]['weight']:
            continue  # Triangle inequality holds
        
    return tsp_tour


In [6]:
train_datasets = ['datasets/eil101.tsp', 'datasets/eil51.tsp', 'datasets/eil76.tsp', 'datasets/fri26.tsp', 'datasets/gr17.tsp', 'datasets/rat99.tsp', 'datasets/st70.tsp']

for dataset in train_datasets:
    curr_dataset = dataset.split('/')[1][:-4]
    print("Dataset Used: ", curr_dataset)
    file = open(dataset)
    graph = tsplib95.read(file).get_graph()
    tsp_tour = christofides(graph)

    optimal_cost = benchmark.optimal_values[curr_dataset]
    total_cost = tour_cost(tsp_tour, graph)

    print("Final TSP Tour:", tsp_tour)
    print("Final Cost of the TSP Tour:", total_cost)
    print("Optimal Cost of the TSP Tour:", optimal_cost)
    print("Approximation Ratio:", approximation_ratio(total_cost, optimal_cost))
    print("--------------------------------------------------")
    

Dataset Used:  eil101
Minimum Cost Perfect Matching Pairs: {(72, 73), (40, 26), (95, 87), (9, 51), (58, 13), (91, 16), (12, 28), (3, 77), (86, 38), (47, 49), (99, 96), (90, 32), (23, 67), (89, 18), (80, 24), (35, 65), (98, 93), (42, 43), (74, 41), (81, 34), (5, 60), (45, 17), (88, 52), (4, 25), (59, 92), (71, 66), (46, 82), (64, 63), (101, 27)}
Final TSP Tour: [1, 50, 76, 77, 3, 29, 24, 80, 54, 55, 25, 4, 56, 39, 67, 23, 75, 74, 41, 22, 73, 72, 21, 40, 26, 58, 13, 94, 6, 89, 18, 60, 5, 84, 17, 45, 8, 46, 82, 48, 47, 49, 36, 19, 11, 64, 63, 90, 32, 10, 62, 88, 31, 70, 30, 20, 66, 71, 65, 35, 9, 51, 81, 34, 78, 33, 79, 68, 12, 28, 101, 27, 53, 97, 87, 42, 43, 15, 57, 2, 95, 98, 37, 100, 91, 16, 86, 38, 14, 44, 61, 85, 93, 59, 92, 99, 96, 83, 7, 52, 69, 1]
Final Cost of the TSP Tour: 726
Optimal Cost of the TSP Tour: 629
Approximation Ratio: 1.1542130365659777
--------------------------------------------------
Dataset Used:  eil51
Minimum Cost Perfect Matching Pairs: {(38, 16), (43, 24), 